In [93]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import torch 
import os 
import cv2 
import tensorflow as tf
from keras.regularizers import l2
from keras.models import Sequential
import time
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.core import Lambda, Flatten, Dense
# Import Keras and other Deep Learning dependencies
from keras.models import Sequential
import time
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
import seaborn as sns
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from sklearn.preprocessing import LabelBinarizer
from keras.optimizers import *
from keras.engine.topology import Layer
from keras import backend as K
from keras.regularizers import l2
K.set_image_data_format('channels_last')
import cv2
import os
from skimage import io
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf


import numpy.random as rng
from sklearn.utils import shuffle


# SIAMESE NETWORK 

In this notebook I will implement a siames network in tensorflow/keras and try it in omniglot dataset 

In [94]:
data_path = os.path.join('../../omniglot')
train_folder = os.path.join(data_path,'images_background')
valpath = os.path.join(data_path,'images_evaluation')

In [95]:
base_class_name = 'character'

In [96]:
img = cv2.imread(train_folder + '/Sanskrit/character11/0861_06.png')

In [97]:
print("Each image in the data set has a same of {0}".format(img.shape))
flattened_img = img.flatten()

print("The number of features in any image from the data set are: {0}".format(flattened_img.shape[0]))

Each image in the data set has a same of (105, 105, 3)
The number of features in any image from the data set are: 33075


In [98]:
def gen_class_names(base_class_name):
    classes = []
    for i in range(1,21):
        if i < 10:
            classes.append("{0}0{1}".format(base_class_name, i))
        else:
            classes.append("{0}{1}".format(base_class_name, i))
    return classes

classes = gen_class_names(base_class_name)

In [99]:
# ONE-HOT ENCODING 

def generate_one_hot_encoding(classes):
    """
    generate one-hot encoding for each classes
    ex character01 = [1,0,0,0,...,0]
    """
    encoder = LabelBinarizer()
    transfomed_labels = encoder.fit_transform(classes)
    return transfomed_labels

In [100]:
labels = generate_one_hot_encoding(classes)

In [101]:
def initialize_weights(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)



def initialize_bias(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

## SIAMESE NETWORK


In [102]:
import tensorflow as tf
#one_one_conv = tf.keras.layers.Conv2D(filters=512,kernel_size=1,padding = 'same',activation='relu',input_shape= CONV5_1_SHAPE, name = 'I_level_conv')(feature_conv5)
def siamese_model(input_shape):
    
    input1 = tf.keras.layers.Input(shape = input_shape, name = "first_image")
    input2 = tf.keras.layers.Input(shape = input_shape, name = "second_image")
    
    #first convolutional block
    
    conv64 = tf.keras.layers.Conv2D(filters = 64,kernel_size = 10,activation='relu',
            kernel_initializer = tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
            input_shape=input_shape,kernel_regularizer=l2(2e-4),
            name = "conv64_10x10")(input1)
    
    # maxpooling    
    maxpool = tf.keras.layers.MaxPooling2D(name = 'first_maxpooling')(conv64)
    
    #second convolutional block 
    
    conv128 = tf.keras.layers.Conv2D(filters = 128,kernel_size = 7,activation='relu',
            kernel_initializer = tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
            bias_initializer=tf.keras.initializers.RandomNormal(mean = 0.5, stddev = 0.01),
            kernel_regularizer=l2(2e-4),
            name = "conv128_7x7")(maxpool)
    
    # maxpooling
    
    maxpool = tf.keras.layers.MaxPooling2D(name = 'second_maxpooling')(conv128)
    
    #third conovolutional block 

    conv128_4 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 4, activation='relu',
                kernel_initializer=tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
                bias_initializer=tf.keras.initializers.RandomNormal(mean = 0.5, stddev = 0.01),
                kernel_regularizer=l2(2e-4),name = 'conv128_4X4')(maxpool)
    
    # maxpool 
    maxpool = tf.keras.layers.MaxPooling2D(name = 'third_maxpooling')(conv128_4)
    
    conv256 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 4, activation='relu',
              kernel_initializer=tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
              bias_initializer=tf.keras.initializers.RandomNormal(mean = 0.5, stddev = 0.01),kernel_regularizer=l2(2e-4),
              name = 'conv256')(maxpool)
    
    
    flatten = tf.keras.layers.Flatten()(conv256)
    
    dense_layer = tf.keras.layers.Dense(4096,activation = 'sigmoid',kernel_regularizer=l2(1e-3),
                kernel_initializer=tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
                bias_initializer=tf.keras.initializers.RandomNormal(mean = 0.5, stddev = 0.01))(flatten)
    
    
    #################################### secondo nn
    
        #first convolutional block
    
    conv64_2 = tf.keras.layers.Conv2D(filters = 64,kernel_size = 10,activation='relu',
            kernel_initializer = tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
            input_shape=input_shape,kernel_regularizer=l2(2e-4),
            name = "conv64_10x10_2")(input2)
    
    # maxpooling    
    maxpool_2 = tf.keras.layers.MaxPooling2D(name = 'first_maxpooling_2')(conv64_2)
    
    #second convolutional block 
    
    conv128_2 = tf.keras.layers.Conv2D(filters = 128,kernel_size = 7,activation='relu',
            kernel_initializer = tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
            bias_initializer=tf.keras.initializers.RandomNormal(mean = 0.5, stddev = 0.01),
            kernel_regularizer=l2(2e-4),
            name = "conv128_7x7_2")(maxpool_2)
    
    # maxpooling
    
    maxpool_2 = tf.keras.layers.MaxPooling2D(name = 'second_maxpooling_2')(conv128_2)
    
    #third conovolutional block 

    conv128_4_2 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 4, activation='relu',
                kernel_initializer=tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
                bias_initializer=tf.keras.initializers.RandomNormal(mean = 0.5, stddev = 0.01),
                kernel_regularizer=l2(2e-4),name = 'conv128_4X4_2')(maxpool_2)
    
    # maxpool 
    maxpool_2 = tf.keras.layers.MaxPooling2D(name = 'third_maxpooling_2')(conv128_4_2)
    
    conv256_2 = tf.keras.layers.Conv2D(filters = 256, kernel_size = 4, activation='relu',
              kernel_initializer=tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
              bias_initializer=tf.keras.initializers.RandomNormal(mean = 0.5, stddev = 0.01),kernel_regularizer=l2(2e-4),
              name = 'conv256_2')(maxpool_2)
    
    
    flatten_2 = tf.keras.layers.Flatten()(conv256_2)
    
    dense_layer_2 = tf.keras.layers.Dense(4096,activation = 'sigmoid',kernel_regularizer=l2(1e-3),
                kernel_initializer=tf.keras.initializers.RandomNormal(mean = 0.0, stddev = 0.01),
                bias_initializer=tf.keras.initializers.RandomNormal(mean = 0.5, stddev = 0.01))(flatten_2)
    
    
    
    
    encoder_l = tf.keras.models.Model(inputs = [input1],outputs = [dense_layer])
    encoder_r = tf.keras.models.Model(inputs = [input2],outputs = [dense_layer_2])
    
    
    
    #### CALCULATE DIFFERENCE 
    
    L1_layer =L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([dense_layer, dense_layer_2])
    
    prediction = tf.keras.layers.Dense(1,activation='sigmoid',
                 bias_initializer=tf.keras.initializers.RandomNormal(mean = 0.5, stddev = 0.01))(L1_distance)
    
    
    siamese_net = tf.keras.models.Model(inputs=[input1,input2],outputs=prediction)
    
    return siamese_net
    
    
    
    
    
    
    
    
    

In [103]:
sm_net = siamese_model((105,105,1))

AttributeError: 'Node' object has no attribute 'output_masks'

In [104]:
from keras import backend as K

In [115]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu',kernel_initializer= 'random_normal', input_shape=input_shape,kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',kernel_initializer= 'random_normal',kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu',kernel_initializer= 'random_normal',kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu',kernel_initializer= 'random_normal',kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',kernel_initializer= 'random_normal',kernel_regularizer=l2(1e-3)))
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    return siamese_net

In [116]:
T = get_siamese_model((105,105,1))

In [117]:
T.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
sequential_10 (Sequential)      (None, 4096)         38947648    input_19[0][0]                   
                                                                 input_20[0][0]                   
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 4096)         0           sequential_10[1][0]        

In [119]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)